# **Stacking from scratch**

Berikan penjelasan terkait Stacking!

**Stacking** adalah membuat model baru yang belajar dari kesalahan dan kekuatan model-model lain untuk membuat keputusan yang lebih baik

Source : https://www.geeksforgeeks.org/machine-learning/stacking-in-machine-learning/
Youtube : https://www.youtube.com/watch?v=a4IS1Ai7GCI

In [12]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

**Pendefinisian class stack**

Bertujuan untuk membuat kerangka kerja ensemble learning berbasis stacking dan blending secara modular dan fleksibel. Class ini memungkinkan pengguna untuk menggabungkan beberapa model dasar (base learners) dan satu model meta (final estimator) untuk menghasilkan prediksi akhir yang lebih akurat. Dengan mengatur parameter seperti metode cross-validation, penggunaan blending, serta paralelisasi proses, class ini memberikan kontrol penuh kepada pengguna dalam menerapkan teknik stacking sesuai kebutuhan. Kegunaan utamanya adalah sebagai alat bantu untuk mengimplementasikan ensemble model dari nol (from scratch) tanpa tergantung pada fungsi otomatis dari pustaka seperti scikit-learn, sehingga cocok untuk pembelajaran konsep maupun eksperimen lanjutan dalam machine learning.

Buat class bernama Stack yang berisi <br>

Attribute:


Method:


In [3]:
# Definisikan class stack
class Stack(object):

  def __init__(self):
        self.item = []
  def push(self, item = ''):

        """

        :param item: Accepts item as Parameter
        :return: None
        """
        self.item.append(item)
        pass

  def pop(self):
        """
        This will remove Last item
        :return: None
        """
        self.item.pop()
        pass

  def peek(self):
        """
        Allows us to see the Last elements
        :return: Last item
        """
        if self.item:
            return self.item[-1]
        else:
            return None

  def size(self):
        if self.item:
            return len(self.item)
        else:
            return None

  def isempty(self):

        """
        tells Whether the stack is Empty or not
        :return: Bool Value
        """
        if self.item == []:
            return True
        else:
            return False

if __name__ == "__main__":
    stack = Stack()

    stack.push("1")
    stack.push('2')
    print(stack.size())
    print(stack.peek())


    stack.pop()
    print(stack.size())
    print(stack.peek())

    print(stack.isempty())

2
2
1
1
False


### **Upload datasets**

In [4]:
# Load dataset iris dari sklearn
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
glass_df = iris.frame

# Pisahkan fitur dan target
X1 = glass_df.drop(columns=['target'])
y1 = glass_df['target']

# Encode target (optional, untuk case multiclass stacking)

# Split data menjadi train dan test
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=0)

# **Model training and evaluation of the obtained results**
Both in the case of classification and regression, stacking and blending showed the same and not the best results. As a rule, this situation occurs for two reasons: given that metadata is based on predictions of basic models, the presence of weak basic models can reduce the accuracy of stronger ones which will reduce the final prediction as a whole. Also a small amount of training data often leads to overfitting which in turn reduces the accuracy of predictions.

In this case the problem can be partially solved by setting stack_method='predict_proba' when each basic classifier outputs class membership probabilities instead of the classes themselves which can help increase accuracy in the case of non-mutually exclusive classes. Also this method works better with noise in the data. As you can see this method has significantly increased the accuracy of the model. With the right selection of models and hyperparameters the accuracy will be even higher.

Most often stacking shows slightly better results than blending due to the use of k-fold cross-validation but usually the difference is noticeable only on a large amount of data.

In [13]:
# StackingClassifier dan Blending Classifier
base_estimators = [
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('dt', DecisionTreeClassifier(max_depth=4, random_state=42)),
    ('lr', LogisticRegression(solver='lbfgs', multi_class='auto', random_state=42))
]

meta_estimator = LogisticRegression(solver='lbfgs', multi_class='auto', random_state=42)

stacking_model_proba = StackingClassifier(
    estimators=base_estimators,       # ✅ tanpa underscore
    final_estimator=meta_estimator,
    cv=5,
    stack_method='predict_proba',
    n_jobs=-1
)


**StackingClassifier (scikit-learn)**


* Merupakan implementasi resmi stacking untuk klasifikasi dalam scikit-learn.

* Mempermudah proses ensemble dengan base learners dan final estimator (meta learner) dalam satu objek.

* Sudah menangani cross-validation secara internal sehingga mengurangi risiko data leakage.

* Mendukung parameter passthrough=True jika ingin menggabungkan fitur asli dengan meta-features.

* Cocok untuk digunakan dalam pipeline dan produksi karena stabil dan teruji.

In [22]:
# StackingClassifier (scikit-learn)
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
# sk_estimators
sk_estimators = [
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('dt', DecisionTreeClassifier(max_depth=4, random_state=42)),
    ('lr', LogisticRegression(solver='lbfgs',  random_state=42))
]

meta_estimator = LogisticRegression(solver='lbfgs',  random_state=42)

# stack_methods
stack_methods = ['auto', 'predict_proba', 'predict']


for i, method in enumerate(stack_methods):
  print(f"({i+1}) stack_method = '{method}'")

  stacking_model = StackingClassifier(
        estimators=sk_estimators,
        final_estimator=meta_estimator,
        cv=5,
        stack_method=method,
        n_jobs=-1
    )

  scores = cross_val_score(stacking_model, X1_train, y1_train, cv=5, scoring='accuracy')
  print(f"Mean CV Accuracy: {np.mean(scores):.4f}")

# model terbaik (opsional)
best_model = StackingClassifier(
    estimators=sk_estimators,
    final_estimator=meta_estimator,
    cv=5,
    stack_method='predict_proba',
    n_jobs=-1
)

best_model.fit(X1_train, y1_train)
print("\nData Test Accuracy:", best_model.score(X1_test, y1_test))

(1) stack_method = 'auto'
Mean CV Accuracy: 0.9462
(2) stack_method = 'predict_proba'
Mean CV Accuracy: 0.9462
(3) stack_method = 'predict'
Mean CV Accuracy: 0.9549

Data Test Accuracy: 0.9736842105263158
